### 로드

In [1]:
# %pip install chromadb sentence_transformers gdown dotenv openai

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import chromadb
from sentence_transformers import SentenceTransformer
import torch
import os
from chromadb.utils import embedding_functions


# ---------------------------------------------------------
# 1. 설정 (GPU 확인 등)
# ---------------------------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# ---------------------------------------------------------
# 2. 모델 로드 (저장할 때 썼던 그 모델!)
# ---------------------------------------------------------
print("모델 로드 중...")
doc_model = SentenceTransformer("dragonkue/BGE-m3-ko").to(device)

ipc_model = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="text-embedding-3-small",
)

# ---------------------------------------------------------
# 3. ChromaDB 연결 (다운로드 받은 폴더 경로 지정)
# ---------------------------------------------------------
doc_client = chromadb.PersistentClient(path="./doc_db")
doc_collection = doc_client.get_collection(name="patent_claims")

ipc_client = chromadb.PersistentClient(path="./ipc_db")
ipc_collection = ipc_client.get_collection(name="ipc_clean")

print(f"✅ doc 데이터베이스 로드 완료! 총 데이터 수: {doc_collection.count()}개")
print(f"✅ ipc 데이터베이스 로드 완료! 총 데이터 수: {ipc_collection.count()}개")

c:\Users\Playdata\.conda\envs\3rdpro_env\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
모델 로드 중...
✅ doc 데이터베이스 로드 완료! 총 데이터 수: 589049개
✅ ipc 데이터베이스 로드 완료! 총 데이터 수: 69337개


### 검색

In [ ]:
# ---------------------------------------------------------
# 4. Doc DB 확인용 함수 (view)
# ---------------------------------------------------------
def search_doc(query_text, top_k=10):
    # 1) 검색어를 벡터로 변환 (임베딩)
    query_vector = doc_model.encode(query_text).tolist()

    # 2) 벡터로 DB 검색
    results = doc_collection.query(query_embeddings=[query_vector], n_results=top_k)

    # 3) 결과 출력
    print(f"\n🔍 검색어: '{query_text}'")

    for i in range(top_k):
        doc = results["documents"][0][i]
        meta = results["metadatas"][0][i]
        distance = results["distances"][0][i]

        print(f"--- [Rank {i+1}] (거리: {distance:.4f}) ---")
        print(f"특허명: {meta.get('title', '제목없음')}")
        print(f"특허ID: {meta.get('patent_id')} (Claim {meta.get('claim_no')})")
        print(f"내용: {doc}")


search_doc("환자의 신체 부위 내에서 내시경을 위치 파악하기 위한 방법")


🔍 검색어: '환자의 신체 부위 내에서 내시경을 위치 파악하기 위한 방법'
--- [Rank 1] (거리: 0.2531) ---
특허명: 내시경 위치 파악 시스템 및 방법
특허ID: 1020257025260 (Claim 1)
내용: 환자의 신체 부위 내에서 내시경을 위치 파악하기 위한 방법으로서,(a) 일련의 EM(electromagnetic) 데이터를 획득하는 단계;(b) 상기 일련의 EM 데이터에 적어도 부분적으로 기초하여 EM 데이터 기반 경로를 생성하는 단계;(c) 상기 일련의 EM 데이터의 데이터 포인트에 적어도 부분적으로 기초하여, 상기 환자의 신체 부위의 생물학적 모델에 기초한 형상을 갖는 하나 이상의 경로 가설을 식별하는 단계;(d) 최적화 알고리즘을 이용하여 상기 하나 이상의 경로 가설을 상기 EM 데이터 기반 경로에 매핑함으로써 하나 이상의 변형된 경로를 생성하는 단계; 및(e) 상기 하나 이상의 경로 가설 각각과 관련된 확률에 적어도 부분적으로 기초하여 상기 하나 이상의 변형된 경로로부터 변형된 경로를 선택하고, 그 선택된 변형된 경로에 기초하여 상기 내시경의 팁의 위치를 결정하는 단계를 포함하는, 방법.
--- [Rank 2] (거리: 0.3384) ---
특허명: 내시경 위치 파악 시스템 및 방법
특허ID: 1020257025260 (Claim 17)
내용: 환자의 신체 부위 내에서 내시경을 위치 파악하기 위한 시스템으로서,컴퓨터 실행 가능 명령어를 저장하는 메모리; 및상기 내시경과 통신하고 상기 컴퓨터 실행 가능 명령어를 실행하여, (a) 일련의 EM(electromagnetic) 데이터를 획득하고; (b) 상기 일련의 EM 데이터에 적어도 부분적으로 기초하여 EM 데이터 기반 경로를 생성하며; (c) 상기 일련의 EM 데이터의 데이터 포인트에 적어도 부분적으로 기초하여, 상기 환자의 신체 부위의 생물학적 모델에 기초한 형상을 갖는 하나 이상의 경로 가설을 식별하고;  (d) 최적화 알고리즘을 이용하여 상기 하나 이상의 경로 가설을 

In [24]:
# ---------------------------------------------------------
# 5. Doc DB 확인용 함수 (view)
# ---------------------------------------------------------
def search_ipc(query_text, top_k=10):
    # 1) 검색어를 벡터로 변환 (임베딩)
    query_vector = ipc_model(query_text)[0].tolist()

    # 2) 벡터로 DB 검색
    results = ipc_collection.query(query_embeddings=[query_vector], n_results=top_k)

    # 3) 결과 출력
    print(f"\n🔍 검색어: '{query_text}'")

    for i in range(top_k):
        ipc_explain = results["documents"][0][i]
        meta = results["metadatas"][0][i]
        ids = results["ids"][0][i]
        distance = results["distances"][0][i]

        print(f"--- [Rank {i+1}] (거리: {distance:.4f}) ---")
        print(f"코드: {ids} (Path {meta.get('path')})")
        print(f"분류: {meta.get('kind')}")
        print(f"내용: {ipc_explain}")


search_ipc(
    "COMPUTER VISION-BASED ALZHEIMER'S DIAGNOSIS SYSTEM AND METHOD USING SUBJECT'S PUPIL MOVEMENT AND MICROSACCADE"
)


🔍 검색어: 'COMPUTER VISION-BASED ALZHEIMER'S DIAGNOSIS SYSTEM AND METHOD USING SUBJECT'S PUPIL MOVEMENT AND MICROSACCADE'
--- [Rank 1] (거리: 0.9922) ---
코드: G06V10/10 (Path G > G06 > G06V > G06V10/00 > G06V10/10)
분류: 1
내용: PHYSICS COMPUTING CALCULATING OR COUNTING IMAGE OR VIDEO RECOGNITION OR UNDERSTANDING Arrangements for image or video recognition or understanding Image acquisition
--- [Rank 2] (거리: 0.9951) ---
코드: G06T7/23 (Path G > G06 > G06T > G06T7/00 > G06T7/20 > G06T7/22 > G06T7/23)
분류: 3
내용: PHYSICS COMPUTING CALCULATING OR COUNTING IMAGE DATA PROCESSING OR GENERATION, IN GENERAL Image analysis Analysis of motion using block-matching using full search
--- [Rank 3] (거리: 1.0024) ---
코드: G06V (Path G > G06 > G06V)
분류: u
내용: PHYSICS COMPUTING CALCULATING OR COUNTING IMAGE OR VIDEO RECOGNITION OR UNDERSTANDING
--- [Rank 4] (거리: 1.0150) ---
코드: G06V30/14 (Path G > G06 > G06V > G06V30/00 > G06V30/10 > G06V30/14)
분류: 2
내용: PHYSICS COMPUTING CALCULATING OR COUNTING IMAGE OR VIDEO RECOGNI